# Reward functie schrijven voor random game
In dit notebook is gewerkt aan een reward functie, zonder dat er al RL wordt gebruikt. Er is 1 game gespeeld, bestaande uit 9 zetten: 9 containers plaatsen in een yard van 3 bij 3. 

In eerste instantie begonnen we met het berekenen van de reward score van 1 game nadat de 9 container geplaatst waren, dit is hier eerst gedaan. Toen kwamen we erachter dat het voor het RL model handig was per zet rewards en penalty's kon geven, zodat het model per zet kon leren wat wel en niet handig was. Dat is in het tweede deel van dit notebook gedaan.

### Imports

In [1]:
import pandas as pd
import numpy as np
import random

### een 3x3 yard maken

In [2]:
list_containers = [1,1,1,2,2,2,3,3,3]

In [3]:
data = {'left':[0,0,0], 'middle':[0,0,0], 'right':[0,0,0]}
df_env = pd.DataFrame(data)

###  Random invullen van environment

In [4]:
def fill_environment(width, length, lijst_containers, d_env):
    env = d_env.copy()
    for j in range(0,length):
        for i in range(0,width):
            if len(lijst_containers) != 0:
                a = random.choice(lijst_containers)
                env.iloc[j,i] = a
                lijst_containers.remove(a)
    return env

In [5]:
lijst = list_containers.copy()
df_env1 = fill_environment(3,3,lijst, df_env)
df_env1

,left,middle,right
0,2,3,1
1,3,3,1
2,1,2,2


## Score aan einde game berekenen

### Score berekenen functie

In [6]:
def score_env(width,length,filled_env):
    score = 0
    for j in range(0,length):
        for i in range(0,(width-1)):
            if filled_env.iloc[j,i] != filled_env.iloc[j,i+1]:
                score += 1
    return score

In [7]:
score = score_env(3,3,df_env1)
score

4

### game spelen en score krijgen

In [8]:
def play_game(width, length, lijst_containers,lege_env1):
    lege_env = lege_env1.copy()
    # laat paar keer random env creëeren en bereken score
    
    lijst = lijst_containers.copy()
    # random vullen env
    random_env = fill_environment(width, length, lijst, lege_env)
    score = score_env(width,length,random_env)
    return score, random_env

In [9]:
game1 = play_game(3, 3, list_containers,df_env)
game1

(2,
    left  middle  right
 0     2       3      3
 1     1       1      1
 2     2       2      3)

---

## Reward per zet
Voor het RL model zo is het nodig om per zet te kijken, dus per plaatsing van elke container. Daarbij deelt die penalty's en rewards uit. De totale game (alle 9 containers plaatsen) heeft dan een bepaalde score/totale reward. Hieronder wordt stap voor stap deze rewardfunctie opgebouwd en vervolgens een game gespeeld.

De containers kunnen op elke plek op random volgorde geplaatst worden. Nu komen eerst de containers 1 binnen, dan 2 en dan de 3en. Dus de volgorde containers is nog niet random

### Penalty's en Rewards uitdelen per zet

In [10]:
def fill_and_reward(lijst_containers, dfenv):
    env = dfenv.copy()
    
    list_ji = [[0,0],[0,1],[0,2],[1,0],[1,1],[1,2],[2,0],[2,1],[2,2]]
    aantal_containers = len(lijst_containers)
    
    penalty_score = 0
    reward_score = 0
    
    for a in range(0,aantal_containers):
        ji = random.choice(list_ji)
        j = ji[0] #row index
        i = ji[1] #column index
        
        #Penalty's:
        # zelfde getal in andere rij plaatsen:
        foute_rij = False
        for row in range(0,3):
            if row != j and lijst_containers[a] in env.iloc[row,0:3].to_list() and lijst_containers[a] not in env.iloc[j,0:3].to_list():
                foute_rij = True
        if foute_rij == True:
            penalty_score -= 1
        # middelste leeg laten
        if i == 0 and env.iloc[j,1] == 0 and env.iloc[j,2] != 0:
            penalty_score -= 3
        if i == 2 and env.iloc[j,1] == 0 and env.iloc[j,0] != 0:
            penalty_score -= 3
        # rechts of links naast ander getal
        if i > 0 and env.iloc[j,i-1] != lijst_containers[a] and env.iloc[j,i-1] != 0:
            penalty_score -= 1
        if i < 2 and env.iloc[j,i+1] != lijst_containers[a] and env.iloc[j,i+1] != 0:
            penalty_score -= 1
        
        #Rewards:
        # naast zelfde getal
        if i > 0 and env.iloc[j,i-1] == lijst_containers[a]:
            reward_score += 2
        if i < 2 and env.iloc[j,i+1] == lijst_containers[a]:
            reward_score += 2
        # in het midden van lege rij plaatsen
        if env.iloc[j,0:3].to_list() == [0,0,0] and i == 1:
            reward_score += 1
        # rij van 3 compleet maken:

        #Plaatsen
        env.iloc[j,i] = lijst_containers[a]
        
        #Reward: rij van 3 compleet maken
        if env.iloc[j,0:3].to_list() == ([1,1,1] or [2,2,2] or [3,3,3]):
            reward_score += 2
        
        print(env)
        print("")
        print(reward_score)
        print(penalty_score)
        print("")
        
        #game over
        if i == 1 and env.iloc[j,0] != 0 and env.iloc[j,2] != 0:
            print("game over")
            print("")
        
        list_ji.remove(ji)
        
    total_reward = reward_score + penalty_score
    return env, total_reward

Hieronder een voorbeeld. Je ziet daar hoe per zet de reward of penalty score veranderd. Aan het einde zie je dan de eindindeling met totale reward waarde van die game

In [11]:
#df_env_c = df_env.copy()
fill_and_reward(list_containers, df_env)

   left  middle  right
0     0       0      1
1     0       0      0
2     0       0      0

0
0

   left  middle  right
0     0       0      1
1     0       0      1
2     0       0      0

0
-1

   left  middle  right
0     0       0      1
1     0       0      1
2     0       1      0

1
-2

   left  middle  right
0     0       0      1
1     0       2      1
2     0       1      0

1
-3

   left  middle  right
0     0       0      1
1     0       2      1
2     0       1      2

1
-5

   left  middle  right
0     0       0      1
1     0       2      1
2     2       1      2

1
-6

   left  middle  right
0     3       0      1
1     0       2      1
2     2       1      2

1
-9

   left  middle  right
0     3       0      1
1     3       2      1
2     2       1      2

1
-11

   left  middle  right
0     3       3      1
1     3       2      1
2     2       1      2

3
-12

game over



(   left  middle  right
 0     3       3      1
 1     3       2      1
 2     2       1      2,
 -9)

# Conclusie
De score van de game hierboven is -9. Dat is een negatieve score dus niet heel goed. Je ziet in het eindresultaat dat er maar 2 containers goed naast elkaar geplaatst zijn, dus dit is niet heel gek. Daarnaast wordt er iets gedaan wat niet mag (er wordt een contianer tussen 2 containers in geplaatst), dus hiervoor heeft die ook een hoge penalty gekregen

de game mag eigenlijk niet containers in het midden plaatsen als daar links en rechts al eentje staat. Voor nu zit er een penalty op, zodat de score lager is als die iets doet wat niet mag. Als het goed is kan de agent bij het RL model dan leren dat dat niet gunstig is en zal dat steeds minder gebeuren. Een andere optie is er een restrictie op zetten.

De volgorde van hoe de containers geplaatst worden is in dit voorbeeld vastgezet. Dit wordt aangepast voor de environment van het RL model. Verder zal er nog veel veranderen aan de reward functie in de volgende stappen van het onderzoek. Dit is een beginnetje en helpt om te begrijpen wat er moet gebeuren bij het vullen.